In [1]:
# Import libraries
import pandas as pd
import plotly.graph_objs as go 
import plotly.express as px 
import dash
from dash import Dash, dcc, html, Input, Output

In [2]:
# Assigning variable to csv path
file_path = '2022-2023.csv'

In [3]:
# Reading in csv file as Pandas DataFrame
init_data_df = pd.read_csv(file_path)
init_data_df.head()

,CRASH_RECORD_ID,CRASH_DATE,Weekday,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,ROADWAY_SURFACE_COND,CRASH_TYPE,...,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,CRASH_TIME
0,9f33be6bff3923ab957886c9cafc6c097322c46d385f1b...,2023-12-31 23:50:00,Sunday,30,NO CONTROLS,RAIN,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,WET,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,23,1,12,41.762206,-87.613072,"['12/31/2023', '11:50:00', 'PM']"
1,2548b7bae8e53a941f28d18ccbe0f444cc9bf3b2c3eaa1...,2023-12-31 23:30:00,Sunday,5,NO CONTROLS,UNKNOWN,UNKNOWN,PARKED MOTOR VEHICLE,UNKNOWN,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,23,1,12,41.949844,-87.646875,"['12/31/2023', '11:30:00', 'PM']"
2,a83a8b4b3c0f2ae287fabeeb70066a1662ee311b394bff...,2023-12-31 23:30:00,Sunday,25,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DRY,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,23,1,12,41.754100,-87.583518,"['12/31/2023', '11:30:00', 'PM']"
3,f3be89d0340e4c18cc21acf686ea99ba028fcbff311460...,2023-12-31 23:29:00,Sunday,30,NO CONTROLS,CLEAR,DARKNESS,FIXED OBJECT,DRY,INJURY AND / OR TOW DUE TO CRASH,...,INCAPACITATING INJURY,1.0,0.0,1.0,23,1,12,41.846412,-87.694857,"['12/31/2023', '11:29:00', 'PM']"
4,2fd2bc0bc3f724fc9580cdfa77446aec2d59226b917b10...,2023-12-31 23:25:00,Sunday,30,TRAFFIC SIGNAL,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,DRY,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,23,1,12,41.928688,-87.642485,"['12/31/2023', '11:25:00', 'PM']"


In [4]:
init_data_df['TRAFFIC_CONTROL_DEVICE'].unique()

array(['NO CONTROLS', 'TRAFFIC SIGNAL', 'STOP SIGN/FLASHER', 'OTHER',
       'UNKNOWN', 'OTHER WARNING SIGN', 'YIELD',
       'PEDESTRIAN CROSSING SIGN', 'OTHER REG. SIGN', 'RR CROSSING SIGN',
       'RAILROAD CROSSING GATE', 'OTHER RAILROAD CROSSING', 'SCHOOL ZONE',
       'POLICE/FLAGMAN', 'FLASHING CONTROL SIGNAL', 'NO PASSING',
       'DELINEATORS', 'BICYCLE CROSSING SIGN'], dtype=object)

In [5]:
delete_values = ['UNKNOWN', 'OTHER', 'OTHER REG. SIGN']
mask = init_data_df['TRAFFIC_CONTROL_DEVICE'].isin(delete_values)
filtered_init_data_df = init_data_df[~mask]
len(filtered_init_data_df)

204364

In [6]:
# Deleting rows with no longitude or latitude values
clean_filtered_df = filtered_init_data_df.dropna(subset=['LATITUDE', 'LONGITUDE', 'MOST_SEVERE_INJURY'])
len(clean_filtered_df)

201999

In [7]:
clean_filtered_df['MOST_SEVERE_INJURY'].unique()

array(['NO INDICATION OF INJURY', 'INCAPACITATING INJURY',
       'REPORTED, NOT EVIDENT', 'NONINCAPACITATING INJURY', 'FATAL'],
      dtype=object)

In [8]:
delete_values = ['NO INDICATION OF INJURY', 'REPORTED, NOT EVIDENT', 'NONINCAPACITATING INJURY']
mask = clean_filtered_df['MOST_SEVERE_INJURY'].isin(delete_values)
final_data = clean_filtered_df[~mask]
print(len(final_data))
final_data.head()

3906


,CRASH_RECORD_ID,CRASH_DATE,Weekday,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,ROADWAY_SURFACE_COND,CRASH_TYPE,...,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,CRASH_TIME
3,f3be89d0340e4c18cc21acf686ea99ba028fcbff311460...,2023-12-31 23:29:00,Sunday,30,NO CONTROLS,CLEAR,DARKNESS,FIXED OBJECT,DRY,INJURY AND / OR TOW DUE TO CRASH,...,INCAPACITATING INJURY,1.0,0.0,1.0,23,1,12,41.846412,-87.694857,"['12/31/2023', '11:29:00', 'PM']"
48,f461cbbdaa5a6fb58d8d5f629cf5d6d4a4522cb12060db...,2023-12-31 19:55:00,Sunday,30,NO CONTROLS,CLOUDY/OVERCAST,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,WET,INJURY AND / OR TOW DUE TO CRASH,...,INCAPACITATING INJURY,1.0,0.0,1.0,19,1,12,41.976253,-87.671648,"['12/31/2023', '07:55:00', 'PM']"
300,48ea3285595a956cd54c213bebd0aab7f3dc9fa933635f...,2023-12-30 20:34:00,Saturday,35,TRAFFIC SIGNAL,CLOUDY/OVERCAST,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,DRY,INJURY AND / OR TOW DUE TO CRASH,...,INCAPACITATING INJURY,1.0,0.0,1.0,20,7,12,41.992473,-87.655312,"['12/30/2023', '08:34:00', 'PM']"
363,3d2517b79bdb8ee4287258a5170457087ffa620aa8815c...,2023-12-30 15:45:00,Saturday,30,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DRY,INJURY AND / OR TOW DUE TO CRASH,...,INCAPACITATING INJURY,1.0,0.0,1.0,15,7,12,41.655293,-87.543661,"['12/30/2023', '03:45:00', 'PM']"
438,ee6fabca3f4dd3fa9a422604693f1dfb6e80f7bb0be54a...,2023-12-30 10:50:00,Saturday,30,NO CONTROLS,CLEAR,DAYLIGHT,REAR TO REAR,DRY,INJURY AND / OR TOW DUE TO CRASH,...,INCAPACITATING INJURY,1.0,0.0,1.0,10,7,12,41.965345,-87.777327,"['12/30/2023', '10:50:00', 'AM']"


In [9]:
final_data['CRASH_DATE'] = pd.to_datetime(final_data['CRASH_DATE'])
final_data.head()

C:\Users\lnfme\AppData\Local\Temp\ipykernel_21068\2479782626.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['CRASH_DATE'] = pd.to_datetime(final_data['CRASH_DATE'])


,CRASH_RECORD_ID,CRASH_DATE,Weekday,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,ROADWAY_SURFACE_COND,CRASH_TYPE,...,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,CRASH_TIME
3,f3be89d0340e4c18cc21acf686ea99ba028fcbff311460...,2023-12-31 23:29:00,Sunday,30,NO CONTROLS,CLEAR,DARKNESS,FIXED OBJECT,DRY,INJURY AND / OR TOW DUE TO CRASH,...,INCAPACITATING INJURY,1.0,0.0,1.0,23,1,12,41.846412,-87.694857,"['12/31/2023', '11:29:00', 'PM']"
48,f461cbbdaa5a6fb58d8d5f629cf5d6d4a4522cb12060db...,2023-12-31 19:55:00,Sunday,30,NO CONTROLS,CLOUDY/OVERCAST,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,WET,INJURY AND / OR TOW DUE TO CRASH,...,INCAPACITATING INJURY,1.0,0.0,1.0,19,1,12,41.976253,-87.671648,"['12/31/2023', '07:55:00', 'PM']"
300,48ea3285595a956cd54c213bebd0aab7f3dc9fa933635f...,2023-12-30 20:34:00,Saturday,35,TRAFFIC SIGNAL,CLOUDY/OVERCAST,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,DRY,INJURY AND / OR TOW DUE TO CRASH,...,INCAPACITATING INJURY,1.0,0.0,1.0,20,7,12,41.992473,-87.655312,"['12/30/2023', '08:34:00', 'PM']"
363,3d2517b79bdb8ee4287258a5170457087ffa620aa8815c...,2023-12-30 15:45:00,Saturday,30,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DRY,INJURY AND / OR TOW DUE TO CRASH,...,INCAPACITATING INJURY,1.0,0.0,1.0,15,7,12,41.655293,-87.543661,"['12/30/2023', '03:45:00', 'PM']"
438,ee6fabca3f4dd3fa9a422604693f1dfb6e80f7bb0be54a...,2023-12-30 10:50:00,Saturday,30,NO CONTROLS,CLEAR,DAYLIGHT,REAR TO REAR,DRY,INJURY AND / OR TOW DUE TO CRASH,...,INCAPACITATING INJURY,1.0,0.0,1.0,10,7,12,41.965345,-87.777327,"['12/30/2023', '10:50:00', 'AM']"


In [10]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3906 entries, 3 to 219117
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   CRASH_RECORD_ID          3906 non-null   object        
 1   CRASH_DATE               3906 non-null   datetime64[ns]
 2   Weekday                  3906 non-null   object        
 3   POSTED_SPEED_LIMIT       3906 non-null   int64         
 4   TRAFFIC_CONTROL_DEVICE   3906 non-null   object        
 5   WEATHER_CONDITION        3906 non-null   object        
 6   LIGHTING_CONDITION       3906 non-null   object        
 7   FIRST_CRASH_TYPE         3906 non-null   object        
 8   ROADWAY_SURFACE_COND     3906 non-null   object        
 9   CRASH_TYPE               3906 non-null   object        
 10  DAMAGE                   3906 non-null   object        
 11  PRIM_CONTRIBUTORY_CAUSE  3906 non-null   object        
 12  NUM_UNITS                3906 non-nul

In [11]:
final_data['CATEGORIES'] = ''

final_data.loc[(final_data['MOST_SEVERE_INJURY'] == 'FATAL') & (final_data['TRAFFIC_CONTROL_DEVICE'] != 'NO CONTROLS'), 'CATEGORIES'] = 'FATAL with Device'
final_data.loc[(final_data['MOST_SEVERE_INJURY'] == 'FATAL') & (final_data['TRAFFIC_CONTROL_DEVICE'] == 'NO CONTROLS'), 'CATEGORIES'] = 'FATAL without Device'
final_data.loc[(final_data['MOST_SEVERE_INJURY'] == 'INCAPACITATING INJURY') & (final_data['TRAFFIC_CONTROL_DEVICE'] != 'NO CONTROLS'), 'CATEGORIES'] = 'INCAPACITATING INJURY with Device'
final_data.loc[(final_data['MOST_SEVERE_INJURY'] == 'INCAPACITATING INJURY') & (final_data['TRAFFIC_CONTROL_DEVICE'] == 'NO CONTROLS'), 'CATEGORIES'] = 'INCAPACITATING INJURY without Device'

final_data.head()

C:\Users\lnfme\AppData\Local\Temp\ipykernel_21068\1839232896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['CATEGORIES'] = ''


,CRASH_RECORD_ID,CRASH_DATE,Weekday,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,ROADWAY_SURFACE_COND,CRASH_TYPE,...,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,CRASH_TIME,CATEGORIES
3,f3be89d0340e4c18cc21acf686ea99ba028fcbff311460...,2023-12-31 23:29:00,Sunday,30,NO CONTROLS,CLEAR,DARKNESS,FIXED OBJECT,DRY,INJURY AND / OR TOW DUE TO CRASH,...,1.0,0.0,1.0,23,1,12,41.846412,-87.694857,"['12/31/2023', '11:29:00', 'PM']",INCAPACITATING INJURY without Device
48,f461cbbdaa5a6fb58d8d5f629cf5d6d4a4522cb12060db...,2023-12-31 19:55:00,Sunday,30,NO CONTROLS,CLOUDY/OVERCAST,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,WET,INJURY AND / OR TOW DUE TO CRASH,...,1.0,0.0,1.0,19,1,12,41.976253,-87.671648,"['12/31/2023', '07:55:00', 'PM']",INCAPACITATING INJURY without Device
300,48ea3285595a956cd54c213bebd0aab7f3dc9fa933635f...,2023-12-30 20:34:00,Saturday,35,TRAFFIC SIGNAL,CLOUDY/OVERCAST,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,DRY,INJURY AND / OR TOW DUE TO CRASH,...,1.0,0.0,1.0,20,7,12,41.992473,-87.655312,"['12/30/2023', '08:34:00', 'PM']",INCAPACITATING INJURY with Device
363,3d2517b79bdb8ee4287258a5170457087ffa620aa8815c...,2023-12-30 15:45:00,Saturday,30,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DRY,INJURY AND / OR TOW DUE TO CRASH,...,1.0,0.0,1.0,15,7,12,41.655293,-87.543661,"['12/30/2023', '03:45:00', 'PM']",INCAPACITATING INJURY without Device
438,ee6fabca3f4dd3fa9a422604693f1dfb6e80f7bb0be54a...,2023-12-30 10:50:00,Saturday,30,NO CONTROLS,CLEAR,DAYLIGHT,REAR TO REAR,DRY,INJURY AND / OR TOW DUE TO CRASH,...,1.0,0.0,1.0,10,7,12,41.965345,-87.777327,"['12/30/2023', '10:50:00', 'AM']",INCAPACITATING INJURY without Device


In [12]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3906 entries, 3 to 219117
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   CRASH_RECORD_ID          3906 non-null   object        
 1   CRASH_DATE               3906 non-null   datetime64[ns]
 2   Weekday                  3906 non-null   object        
 3   POSTED_SPEED_LIMIT       3906 non-null   int64         
 4   TRAFFIC_CONTROL_DEVICE   3906 non-null   object        
 5   WEATHER_CONDITION        3906 non-null   object        
 6   LIGHTING_CONDITION       3906 non-null   object        
 7   FIRST_CRASH_TYPE         3906 non-null   object        
 8   ROADWAY_SURFACE_COND     3906 non-null   object        
 9   CRASH_TYPE               3906 non-null   object        
 10  DAMAGE                   3906 non-null   object        
 11  PRIM_CONTRIBUTORY_CAUSE  3906 non-null   object        
 12  NUM_UNITS                3906 non-nul

In [13]:
final_data.to_csv('scatter_mapbox_data.csv', index=False)

In [13]:
categories = final_data['CATEGORIES'].unique()
categories

array(['INCAPACITATING INJURY without Device',
       'INCAPACITATING INJURY with Device', 'FATAL with Device',
       'FATAL without Device'], dtype=object)

In [14]:
# Initialize the Dash app
app = dash.Dash(__name__)

In [15]:
# Extract unique years and months from the CRASH_DATE column
# Manually define unique months for correct order in dropdown menu
unique_years = final_data['CRASH_DATE'].dt.year.unique()
unique_months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [16]:
# Create list of dictionaries to give user option to select data for full year
year_options = [{'label': 'All of 2022', 'value': 'All of 2022'},
                {'label': 'All of 2023', 'value': 'All of 2023'}]

In [17]:
# Create the layout using html.Div 
app.layout = html.Div([
    # H1 element for the title
    html.H1("Impact of Traffic Control Devices on Car Accidents",  style={'width': '500px', 'white-space': 'pre-wrap'}),
    html.Div([
        html.Div([
            # Creating dropdown for year for user to select
            html.Label("Select Year:", style={'font-size': '14px'}),
            dcc.Dropdown(
                id='year-dropdown',
                # This creates options to select a year along with a month or a full year (2022 or 2023)
                options=[{'label': str(year), 'value': year} for year in unique_years] + year_options,
                # default value is "All of 2022"
                value=year_options[0]['value'],
                # Styled by using CSS for positioning in the map and font size
                style={'width': '150px', 'font-size': '14px'}
            )
        ], style={'position': 'absolute', 'top': '10px', 'right': '180px'}),
        
        html.Div([
            # Creating dropdown for month for user to select
            html.Label("Select Month:", style={'font-size': '14px'}),
            dcc.Dropdown(
                id='month-dropdown',
                options=[{'label': month, 'value': month} for month in unique_months],
                value=unique_months[0],
                style={'width': '150px', 'font-size': '14px'}
            )
        ], style={'position': 'absolute', 'top': '10px', 'right': '10px'}),
    ], style={'position': 'relative', 'height': '100px'}),    
    # Create a graph for displaying the map
    dcc.Graph(id='map-graph')
])
    

In [18]:
# Define callback to update month dropdown options based on selected year
@app.callback(
    dash.dependencies.Output('month-dropdown', 'options'),
    [dash.dependencies.Input('year-dropdown', 'value')])

# defined a function that takes selected year as input and returns the month optioms
def update_month_options(selected_year):
    if selected_year in ['All of 2022', 'All of 2023']:
        # If "All of 2022" or "All of 2023" is selected, return an empty list for month options
        return []
    else:
        # Return the options for all months if "All of 2022" or "All of 2023" is NOT selected
        return [{'label': month, 'value': month} for month in unique_months]

# Define callback to update the map based on user selection
@app.callback(
    dash.dependencies.Output('map-graph', 'figure'),
    [dash.dependencies.Input('year-dropdown', 'value'),
     dash.dependencies.Input('month-dropdown', 'value')]
)

# Function that takes selected year and selected month as input
def update_map(selected_year, selected_month):
    # Convert month name to numeric values
    month_to_num = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
                    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

    # Check if "All of 2022" or "All of 2023" is selected
    if selected_year in ['All of 2022', 'All of 2023']:
        # Filter the DataFrame for the entire year
        # Splits the selected year string into a list of strings, whitespace is delimiter, selects last element of that list
        filtered_df = final_data[final_data['CRASH_DATE'].dt.year == int(selected_year.split()[-1])]
    else:
        # Convert selected year to integer and filter df based on selected year and month
        selected_year = int(selected_year)
        selected_month_num = month_to_num[selected_month]
        filtered_df = final_data[(final_data['CRASH_DATE'].dt.year == selected_year) & 
                                 (final_data['CRASH_DATE'].dt.month == selected_month_num)]

    # Create map using Plotly Express
    fig = px.scatter_mapbox(filtered_df, lat='LATITUDE', lon='LONGITUDE', zoom=10)

    # Define categories and colors
    categories = sorted(filtered_df['CATEGORIES'].unique())
    colors = ['red', 'blue', 'yellow', 'green']
    
    # Add markers for each category
    for category, color in zip(categories, colors):
        # Filtering based on selected category
        filtered_category_df = filtered_df[filtered_df['CATEGORIES'] == category]
        # Grab count to show in tracing
        count = len(filtered_category_df)
        # Naming the tracing attributes by categories and by the count of those categories
        name = f"{category}: {count}"
        # List comprehension to iterate through all rows of df to pull date and device info
        hover_text = [f"CRASH_DATE: {row['CRASH_DATE']}<br>TRAFFIC_CONTROL_DEVICE: {row['TRAFFIC_CONTROL_DEVICE']}" for _, row in filtered_category_df.iterrows()]
        #Creating the scatter mapbox
        fig.add_scattermapbox(
            lat=filtered_category_df['LATITUDE'],
            lon=filtered_category_df['LONGITUDE'],
            mode='markers',
            name=name,
            marker=dict(color=color, size=10),
            text=hover_text,
            hoverinfo='text'
        )

    # Defining type of map and margins    
    fig.update_layout(mapbox_style="carto-positron")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)